# Work through the reduction of a single dataset

In [1]:
# imports
import os
import glob
from imp import reload

from cosredux import utils as cr_utils
from cosredux import trace as cr_trace

## Setup files
    Copy files from Dropbox to local, working folder
    cd 'working_folder'
    # Darks, if needed
    cp -rp ~/Dropbox/COS-LRG/darksall .
    # Calibs, as needed
    cp -rp ~/Dropbox/COS-LRG/calibfilesmast .
    # Subset of the raw and object calibration files
    mkdir LCYA01010
    cd LCYA01010
    cp -rp ~/Dropbox/COS-LRG/LCYA01010/*rawtag* .
    cp -rp ~/Dropbox/COS-LRG/LCYA01010/lcya01010_asn.fits .
    cp -rp ~/Dropbox/COS-LRG/LCYA01010/lcya01010_trl.fits .
    cp -rp ~/Dropbox/COS-LRG/LCYA01010/lcya01010_j*.fits .

In [2]:
#rdx_path = '/Users/xavier/HST/COS/LRG_Redux/'
rdx_path = '/home/marijana/ReductionCOS/'
science_folder = 'LCYA01010/'
dark_folder = 'darks/'
calib_folder = 'calibfilesmast/'
root_out = 'lcya01010'

## Customize for calcos
    Science frames
    LP2 calib file

### Science frames

In [3]:
cr_utils.modify_rawtag_for_calcos(rdx_path+science_folder)

Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01ggq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01fyq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01ggq_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01g0q_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01gjq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01fyq_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01gjq_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01g0q_rawtag_a.fits


### Calibs -- May not need to repeat this (i.e. if done before)

In [4]:
cr_utils.modify_LP2_1dx_calib(rdx_path+calib_folder)

----

## Run calcos

### Launch pyraf
    cd ureka
    xgterm (or xterm)
    export lref=/home/marijana/ReductionCOS/calibfilesmast/
    # Working in xgterm window
    ur_setup
    mkiraf #?
    pyraf

### Execute
    cd folder_with_data  (e.g. LCYA01010)
    stsdas
    hst_calib
    hstcos
    #
    calcos [dataset prefix]_asn.fits (e.g. calcos lcya01010_asn.fits)

#### Crashing and rerunning
    If you have to rerun, remove the corrtag and .tra files

----

## Coadd corrtag files

In [3]:
corrtag_files_a = glob.glob(rdx_path+science_folder + '*_corrtag_a.fits')
corrtag_files_b = glob.glob(rdx_path+science_folder + '*_corrtag_b.fits')

In [4]:
corrtag_files_a

['/home/marijana/ReductionCOS/LCYA01010/lcya01ggq_corrtag_a.fits',
 '/home/marijana/ReductionCOS/LCYA01010/lcya01g0q_corrtag_a.fits',
 '/home/marijana/ReductionCOS/LCYA01010/lcya01fyq_corrtag_a.fits',
 '/home/marijana/ReductionCOS/LCYA01010/lcya01gjq_corrtag_a.fits']

In [6]:
_ = cr_utils.coadd_bintables(corrtag_files_a, outfile=rdx_path+science_folder+root_out+'_coaddcorr_a.fits')
_ = cr_utils.coadd_bintables(corrtag_files_b, outfile=rdx_path+science_folder+root_out+'_coaddcorr_b.fits')

Wrote outfile /home/marijana/ReductionCOS/LCYA01010/lcya01010_coaddcorr_a.fits
Wrote outfile /home/marijana/ReductionCOS/LCYA01010/lcya01010_coaddcorr_b.fits


## Traces

In [10]:
reload(cr_trace)
traces_a=cr_trace.traces(rdx_path+science_folder+root_out+'_coaddcorr_a.fits', rdx_path+calib_folder, 'FUVA', clobber=True)
traces_b=cr_trace.traces(rdx_path+science_folder+root_out+'_coaddcorr_b.fits', rdx_path+calib_folder, 'FUVB', clobber=True)

Updated trace for segment=FUVA in /home/marijana/ReductionCOS/calibfilesmast/x6q17586l_1dx.fits
Updated trace for segment=FUVB in /home/marijana/ReductionCOS/calibfilesmast/x6q17586l_1dx.fits
